In [1]:
from timeit import default_timer as timer   
import time
import sys
import numpy as np
from pyspark.sql import SparkSession
from pyspark.serializers import PickleSerializer
from pyspark import SparkContext, SparkConf
import numpy as np
import random
from pyswarm import pso



In [2]:

# Create a Spark context object using the configuration

spark = SparkSession.builder.appName("MyApp").config("spark.driver.memory", "28g").config("spark.executor.memory", "80g").config("spark.driver.maxResultSize", "10g").getOrCreate()



23/04/24 17:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
lines_rdd = spark.sparkContext.textFile("file:///gpfs/SHPC_Data/home/aafolab4/ccFraud.csv");
filtered_rdd = lines_rdd.filter(lambda x: x != '"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"')
filtered_rdd = filtered_rdd.map(lambda x: (x))
parsed_rdd = filtered_rdd.map(lambda line: line.split(","))
parsed_rdd = parsed_rdd.map(lambda x : tuple(x))
parsed_rdd = parsed_rdd.map(lambda x : tuple(np.float32(x)))
data_rdd = parsed_rdd.map(lambda x : (int(x[-1]),x[1:8]))
#train_rdd = data_rdd.filter(lambda x : x[0] == 0).map(lambda x: x[1])
#train_labels_rdd =  data_rdd.filter(lambda x : x[0] == 0).map(lambda x: x[0])
#test_rdd = data_rdd.filter(lambda x : x[0] == 1).map(lambda x: x[1])
#test_labels_rdd = data_rdd.filter(lambda x : x[0] == 1).map(lambda x: x[0])

In [4]:
parsed_rdd.take(5)

[(1.0, 1.0, 35.0, 1.0, 3000.0, 4.0, 14.0, 2.0, 0.0),
 (2.0, 2.0, 2.0, 1.0, 0.0, 9.0, 0.0, 18.0, 0.0),
 (3.0, 2.0, 2.0, 1.0, 0.0, 27.0, 9.0, 16.0, 0.0),
 (4.0, 1.0, 15.0, 1.0, 0.0, 12.0, 0.0, 5.0, 0.0),
 (5.0, 1.0, 46.0, 1.0, 0.0, 11.0, 16.0, 7.0, 0.0)]

In [5]:
train_rdd = data_rdd.filter(lambda x : x[0] == 0).map(lambda x: x[1])
train_labels_rdd =  data_rdd.filter(lambda x : x[0] == 0).map(lambda x: x[0])
test_rdd = data_rdd.filter(lambda x : x[0] == 1).map(lambda x: x[1])
test_labels_rdd = data_rdd.filter(lambda x : x[0] == 1).map(lambda x: x[0])

In [6]:
a= train_rdd.collect()
a = np.array(a)
b= np.array(test_rdd.collect()) 

In [7]:
class PSOAANN:
    def __init__(self, n_hidden, c1, c2, w):
        self.n_hidden = n_hidden
       # self.scaler = None
        self.w1 = None
        self.w2 = None
        self.c1 = c1
        self.c2 = c2
        self.w = w
        
   
    def normalize(self, X):
        X_rdd = spark.sparkContext.parallelize(X, numSlices=10000)
        min_vals = X_rdd.reduce(lambda x, y: tuple(map(min, zip(x, y))))
        max_vals = X_rdd.reduce(lambda x, y: tuple(map(max, zip(x, y))))
        X_scaled_rdd = X_rdd.map(lambda x : tuple((np.array(x) - np.array(min_vals)) / (np.array(max_vals) - np.array(min_vals))))
        X_scaled = np.array(X_scaled_rdd.collect())
        return X_scaled
        
    def fit(self, X):
        
        X_scaled = self.normalize(X)
        
        n_input = X_scaled.shape[1]
        #n_output = len(np.unique(y))
        
        # Define bounds for PSO optimization
        lower_bounds = [-5.0] * (n_input * self.n_hidden + self.n_hidden * n_input)
        upper_bounds = [5.0] * (n_input * self.n_hidden + self.n_hidden * n_input)
        bounds = (lower_bounds, upper_bounds)
        
        
        # Define PSO fitness function
        def fitness(params):
            w1 = params[:n_input * self.n_hidden].reshape((n_input, self.n_hidden))
            w2 = params[n_input * self.n_hidden:].reshape((self.n_hidden, n_input))
            
            # Feedforward calculation
            a1 = np.dot(X_scaled, w1)
            a1_rdd = spark.sparkContext.parallelize(a1, numSlices=10000)
            a1_rdd = a1_rdd.map(lambda x: tuple(map(lambda y: 1.0 / (1.0 + np.exp(-y)), x)))
            a1 = np.array(a1_rdd.collect())
            
            X_pred = np.dot(a1, w2)
            X_pred_rdd = spark.sparkContext.parallelize(X_pred, numSlices=10000)
            X_pred_rdd = X_pred_rdd.map(lambda x: tuple(map(lambda y: 1.0 / (1.0 + np.exp(-y)), x)))
            X_pred = np.array(X_pred_rdd.collect())
            
            # Calculate accuracy as fitness score
            #mse = np.sum((X_scaled-X_pred)**2)/(self.n_hidden * n_input)
            mse = np.mean((X_scaled-X_pred)**2)
            #print(mse)
            #y_pred_labels = np.argmax(y_pred, axis=1)
            #score = accuracy_score(y, y_pred_labels)
            return mse
  
        # Run PSO optimization
        mse_mean = np.zeros(1)
        params = np.random.uniform(low=-5,high=5,size=n_input * self.n_hidden * 2)
        for i in range(1):
            params, _ = pso(fitness, *bounds, swarmsize=1, maxiter=1, omega=self.w, phip=self.c1, phig=self.c2)
            mse = fitness(params)
            mse_mean[i] = np.mean(np.array(mse))
            print("Iteration {} gives MSE {}".format(i, mse)) 

             # Retrieve best weights
            self.w1 = params[:n_input * self.n_hidden].reshape((n_input, self.n_hidden))
            self.w2 = params[n_input * self.n_hidden:].reshape((self.n_hidden, n_input))
        print("The final mean mse is "+str(np.mean(mse_mean)))
    def predict(self, X):
        X_scaled = self.normalize(X)
        a1 = np.dot(X_scaled, self.w1)
        a1_rdd = spark.sparkContext.parallelize(a1, numSlices=10000)
        a1_rdd = a1_rdd.map(lambda x: tuple(map(lambda y: 1.0 / (1.0 + np.exp(-y)), x)))
        a1 = np.array(a1_rdd.collect())

        X_pred = np.dot(a1, self.w2)
        X_pred_rdd = spark.sparkContext.parallelize(X_pred, numSlices=10000)
        X_pred_rdd = X_pred_rdd.map(lambda x: tuple(map(lambda y: 1.0 / (1.0 + np.exp(-y)), x)))
        X_pred = np.array(X_pred_rdd.collect())

        #relative error
        relative_error = np.true_divide(np.abs(X_pred - X_scaled),X_scaled)
        #y_pred_labels = np.argmax(y_pred, axis=1)
        return relative_error


In [8]:
psoaann = PSOAANN(n_hidden=6, c1=2, c2=2, w=0.9)
start = time.time()
psoaann.fit(a)
finish = time.time()
print("Training this took ", round(finish-start,3), " secs")
X_pred = psoaann.predict(b)
def final_pred(X_pred,threshold):
    h = np.where(X_pred > threshold ,1, 0)
    pred_class = []
    for i in h:
        if list(i).count(1) == len( list(i)) :
            pred_class.append(1)
        else:
            pred_class.append(0)
    
    return pred_class

pred = np.array(final_pred(b,0.05))

Classification_rate = np.count_nonzero(pred) /len(b)
print("The classification rate is "+str(Classification_rate))

[Stage 3:==========>                                        (1961 + 32) / 10000]23/04/24 17:20:40 WARN TaskSetManager: Stage 3 contains a task of very large size (157 KB). The maximum recommended task size is 100 KB.
[Stage 4:=========>                                         (1924 + 32) / 10000]23/04/24 17:20:55 WARN TaskSetManager: Stage 4 contains a task of very large size (157 KB). The maximum recommended task size is 100 KB.
[Stage 5:==========>                                        (1967 + 32) / 10000]23/04/24 17:21:11 WARN TaskSetManager: Stage 5 contains a task of very large size (157 KB). The maximum recommended task size is 100 KB.
23/04/24 17:22:09 WARN TaskSetManager: Stage 6 contains a task of very large size (100 KB). The maximum recommended task size is 100 KB.
23/04/24 17:23:13 WARN TaskSetManager: Stage 7 contains a task of very large size (108 KB). The maximum recommended task size is 100 KB.
23/04/24 17:24:22 WARN TaskSetManager: Stage 8 contains a task of very larg

Stopping search: maximum iterations reached --> 1


23/04/24 17:26:36 WARN TaskSetManager: Stage 10 contains a task of very large size (100 KB). The maximum recommended task size is 100 KB.
23/04/24 17:27:40 WARN TaskSetManager: Stage 11 contains a task of very large size (108 KB). The maximum recommended task size is 100 KB.


Iteration 0 gives MSE 0.2152761360896253
The final mean mse is 0.2152761360896253
Training this took  493.803  secs


/gpfs/SHPC_Data/home/aafolab4/.conda/envs/myJupyter/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in true_divide


The classification rate is 0.44537544420097513
